# AI Project : Biscuit Factory optimization

### Imports and initialization

In [1]:
import numpy as np
rng = np.random.default_rng()


You also need the ```python-constraint``` library in conda, the ```biscuit_clement``` and ```biscuit_racel``` files, as they contain different implementations of the Roll class.  
We have two implementations of the Roll class because we originally had one, then Clement copied it worked on his copy and Racel modified things on the original one.  
When we tried to merge all the algorithms we had in the end, Clement's algorithms wouldn't work with Racel 's Roll class and the inverse was also true.  
So we decided to keep it that way, we lacked time to try to merge the two Roll classes. In the notebook, the proper Roll class is reimported before each algorithm.

First we did greedy search and backtracking csp algorithms.

### Greedy search

In [2]:
from biscuits_clement import biscuit_types, Roll, defects_list
from constraint import Problem  # from the python-constraint librairy in conda


# Global access within this module for defects_list
def get_defects_between(start, end, defects_list):
    return [defect for defect in defects_list if start <= defect['x'] < end]

# Constraint to ensure no overlapping biscuits
def no_overlap_constraint(*positions):
    sizes = [b.size for b in biscuit_types]  # Access the global biscuit_types
    for i, pos in enumerate(positions[:-1]):
        if pos + sizes[i] > positions[i + 1]:
            return False
    return True

# Constraint to ensure defects are within tolerance
def defects_within_tolerance(*positions):
    for i, pos in enumerate(positions):
        biscuit = biscuit_types[i]  # Access the global biscuit_types
        defects_in_range = get_defects_between(pos, pos + biscuit.size, defects_list)
        defect_counts = {defect_class: 0 for defect_class in biscuit.tolerance.keys()}
        for defect in defects_in_range:
            defect_class = defect['class']
            defect_counts[defect_class] += 1
        if not biscuit.is_valid(defect_counts):
            return False
    return True

# Function to calculate the total value of biscuits on the roll
def calculate_total_value(*positions):
    total_value = 0
    for i, pos in enumerate(positions):
        total_value += biscuit_types[i].value
    return total_value

# Create the problem instance
problem = Problem()

# Variables for the positions of each biscuit type on the roll
roll_size = 500

# Assuming that biscuit_types are sorted by value with the most valuable first
biscuit_types.sort(key=lambda b: -b.value)
for i, biscuit in enumerate(biscuit_types):
    problem.addVariable(f"Biscuit_{i}", range(roll_size - biscuit.size + 1))

# Add constraints to the problem
problem.addConstraint(no_overlap_constraint, [f"Biscuit_{i}" for i in range(len(biscuit_types))])
problem.addConstraint(defects_within_tolerance, [f"Biscuit_{i}" for i in range(len(biscuit_types))])

# Heuristic approach
def greedy_heuristic():
    roll = Roll(roll_size=500)
    positions_filled = [False] * 500  # A simple way to track filled positions

    for biscuit in sorted(biscuit_types, key=lambda b: -b.value):  # Sort by value as a heuristic
        for position in range(roll.roll_size - biscuit.size + 1):
            if not any(positions_filled[position:position + biscuit.size]):  # Check if space is free
                defects_in_range = get_defects_between(position, position + biscuit.size, defects_list)
                defect_counts = {defect_class: 0 for defect_class in biscuit.tolerance.keys()}
                for defect in defects_in_range:
                    defect_class = defect['class']
                    defect_counts[defect_class] += 1

                if biscuit.is_valid(defect_counts):
                    # Fill the positions and add the biscuit to the roll
                    for i in range(position, position + biscuit.size):
                        positions_filled[i] = True  # Mark positions as filled
                    biscuit_dict = {'start': position, 'end': position + biscuit.size, 'biscuit': biscuit}
                    roll.append_biscuits(biscuit_dict)

    return roll


In [3]:
# Run the heuristic and get the Roll object with the best arrangement
best_roll = greedy_heuristic()

# Now you can access the total value and other properties of the best_roll
print(f"The total value of the best roll is: {best_roll.total_value()}")
print(f"The biscuits counts of the best roll is: {best_roll.biscuit_type_count()}")


The total value of the best roll is: 621
The biscuits counts of the best roll is: {8: 44, 5: 1, 4: 14, 2: 1}


Majority of biscuits of size 8 and 4

### Backtracking algorithm

In [4]:
from biscuits_clement import biscuit_types, defects_list, Roll


# Helper function to check if a biscuit can be placed considering defects and overlapping
def is_valid_position(start, biscuit, roll, defects_list):
    # Check for overlap
    for other in roll._biscuits:
        if start < other['end'] and other['start'] < start + biscuit.size:
            return False  # Overlap detected

    # Check defects within the biscuit's range
    defects_in_range = [defect for defect in defects_list if start <= defect['x'] < start + biscuit.size]
    defect_counts = {defect_class: 0 for defect_class in biscuit.tolerance.keys()}
    for defect in defects_in_range:
        defect_class = defect['class']
        defect_counts[defect_class] += 1

    return biscuit.is_valid(defect_counts)


# Backtracking algorithm
def place_biscuits(roll, biscuits, defects_list, position=0):
    if position >= roll.roll_size:  # Reached the end of the roll
        return roll.total_value(), roll._biscuits.copy()

    max_value = 0
    best_arrangement = None

    for biscuit in biscuits:
        if is_valid_position(position, biscuit, roll, defects_list):
            # Create a new biscuit dict to represent the placed biscuit
            biscuit_dict = {'start': position, 'end': position + biscuit.size, 'biscuit': biscuit}
            # Place the biscuit
            roll.append_biscuits(biscuit_dict)  # Now passing a dict object
            # Recurse to the next position
            value, arrangement = place_biscuits(roll, biscuits, defects_list, position + biscuit.size)

    # Consider not placing a biscuit at this position
    value, arrangement = place_biscuits(roll, biscuits, defects_list, position + 1)
    if value > max_value:
        max_value = value
        best_arrangement = arrangement

    return max_value, best_arrangement


In [5]:
# Instantiate a Roll object
roll = Roll(roll_size=500)
# Sort biscuits by value, descending (as a heuristic)
sorted_biscuits = sorted(biscuit_types, key=lambda b: -b.value)

# Find the best arrangement
total_value, arrangement = place_biscuits(roll, sorted_biscuits, defects_list)

# Create a new Roll object with the best arrangement
best_roll = Roll(roll_size=500)
best_roll._biscuits = arrangement  # Directly setting the best arrangement

# Now you can access the total value and other properties of the best_roll
print(f"The total value of the best roll is: {best_roll.total_value()}")
print(f"The biscuits counts of the best roll is: {best_roll.biscuit_type_count()}")


The total value of the best roll is: 643
The biscuits counts of the best roll is: {8: 43, 5: 0, 4: 21, 2: 1}


Majority of biscuits of size 8 and 4 too

We notice same distributions of biscuit types for greedy search and backtracking algorithm. This is because of the similar functionnality of these two algorithms, as they try to arrange for the best arrangement.

### Bogo search

In [6]:
from biscuits_racel import Roll


# bogo optimizer (as in bogo sort : https://en.wikipedia.org/wiki/Bogosort) 
# is an optimizing algorithm that generates a random roll at each iteration and compares it to the best roll found.
# It is different to most random optimizer like random search as it generates a new solution at each iteration, instead of modifying the current best roll.
def bogo(max_iter):
    # initialize the best roll
    best_roll = None
    best_score = float('-inf')    
    # at each iteration we generate a random roll and 
    for _ in range(max_iter):
        roll = Roll(500)
        # randomly fill the roll, we enable biscuit constraint checking so that biscuits must be a biscuit value 
        roll.fill_roll_random(check_biscuit_valid=True)
        v = roll.total_price()
        if v > best_score:
            best_roll = roll
            best_score = v

    return best_roll, best_score

In [7]:
best_r, best_s = bogo(500)
print(f'Best price is : {best_s}, best number of biscuits is :{best_r.number_of_biscuits()}')
print(f'Best food biscuit counts is : {best_r.biscuit_type_count()}')

Best price is : 659, best number of biscuits is :138
Best food biscuit counts is : {8: 25, 5: 19, 4: 32, 2: 15}


Majority of size 8 and 4 biscuits as well, but more biscuits of sizes 2 and 5

### Artificial bee colony algorithm

In [8]:
from bees import Hive, Scout, Worker, Onlooker
from bees import names, generate_new_food, evaluate_roll

def bee_search(obj_func,
               minimize=False,
               n_bees=10,
               n_workers=None,
               n_scouts=1,
               max_iter=1000,
               limit=5):
    # The minimize parameter indicates whether the objective function should be minimized,
    # in which case it should maximize the opposite, or not.
    if minimize:
        def fitness(roll):
            return -1 * obj_func(roll)
    else:
        def fitness(roll):
            return obj_func(roll)

    # initializing the number of worker bees, if None
    if n_workers is None:
        n_workers = n_bees // 2

    # Each worker should be assigned a food source
    # So the number of food source is equal to the number of workers
    n_foods = n_workers

    # initialize the hive
    hive = Hive({
        'scouts': [Scout(fitness, name=rng.choice(names)) for _ in range(0, n_scouts)],
        'onlookers':
            [Onlooker(fitness, name=rng.choice(names)) for _ in range(0, n_bees - n_workers)],
        'workers': [Worker(fitness, name=rng.choice(names)) for _ in range(0, n_workers)]
    })

    # generate an array of random coordinates in the search space
    # of shape [[...] * food_sources_initial]
    initial_foods = generate_new_food(n_foods, food_quantity=limit)

    # Send all workers at the initial food
    for worker, food in zip(hive.get_workers(), initial_foods):
        worker.go_to_food(food)
        worker.dance()

    # Get the initial best food
    best_quality_food = max(hive.get_workers(), key=lambda worker_bee: worker_bee.food.quality).food

    # Algorithm loop
    for i in range(max_iter):
        """Workers phase"""

        for worker in hive.get_workers():
            # We first check if the source has been depleted by the onlookers
            # Indeed we gave the ability for onlooker to bring food to the hive, 
            # so the food source gets depleted by both type of bees.
            # Because of the loop, if we have a food source of quantity value 0,
            # the worker bee has to leave and join the scouts to search for a new food source.
            if worker.should_leave():
                new_scout = worker.leave_food_point()
                hive.get_scouts().append(new_scout)
                hive.get_workers().remove(worker)
                continue

            worker.dance()  # Workers register their food source and give them a food source quality value

            # Search for a better food source around the current food source
            new_solution = worker.look_around()
            # Evaluate food source
            new_solution_evaluation = worker.calculate_nectar(new_solution)
            # If food source is better, discard old food source and select new open
            # Else bring food to the hive, or leave if no more food is available
            if new_solution_evaluation > worker.food.quality:
                worker.food = new_solution
                worker.dance()
                if new_solution_evaluation > best_quality_food.quality:
                    best_quality_food = new_solution
            else:
                worker.bring_food()
                if worker.should_leave():
                    new_scout = worker.leave_food_point()
                    hive.get_scouts().append(new_scout)
                    hive.get_workers().remove(worker)

        """Onlookers phase"""
        # Onlookers choose a food source, among the worker ones.
        # This choice is made by probabilities, 
        # otherwise they'd just choose the best one and never explore around the other ones.

        # Calculate the probability value of the sources
        # with which they are preferred by the onlooker bees
        # First they watch the bees dance
        # choreography is of type (worker, worker.food, food.quality)
        choreography = np.array([worker.dance() for worker in hive.get_workers()])
        # print([c.quality for c in choreography[:, 0]])
        choreography[:, 1] = np.array([dance if dance > float('-inf') else 0 for dance in choreography[:, 1]])
        # convert -inf to 0
        choreography[:, 1] = np.abs(np.nan_to_num(choreography[:, 1]))
        # calculate the sum of all the qualities
        sum_dances = np.sum(choreography[:, 1])
        # for functions that are flat on most of their definition space, sum could be zero
        # we have to check for that condition to prevent having nan values in the probabilities list
        if sum_dances == 0.0:
            probabilities = None
        else:
            # divide each value to get the probability for the onlooker to choose that food source
            probabilities = np.divide(choreography[:, 1], sum_dances).astype('float64')
            # In that case, every food source has the same probability of being chosen which is not optimal.
            # We want to have most likeliness for the most promising source

        for onlooker in hive.get_onlookers():
            # Onlookers choose a food source, then dance.
            onlooker.choose_preferred_source(choreography[:, 0], probabilities)
            onlooker.dance()
            # Onlookers search for a new food source
            new_solution = onlooker.look_around()
            # Evaluate food source
            new_solution_evaluation = onlooker.calculate_nectar(new_solution)
            # If food source is better, discard old food source and select new food source
            if new_solution_evaluation > onlooker.food.quality:
                onlooker.food = new_solution
                onlooker.dance()
                if new_solution_evaluation > best_quality_food.quality:
                    best_quality_food = new_solution
            # Else increase limit counter
            else:
                if onlooker.food.has_food():
                    onlooker.bring_food()
                # If the food is exhausted, leave the food source
                if onlooker.should_leave():
                    onlooker.leave_food_point()

        """Scouts phase"""
        # Scouts search for a new food source around the search space. 
        for scout in hive.get_scouts():
            # Find a new food source
            scout.find_new_food(quantity=limit)
            # Convert back to worker and go on that food source
            new_worker = scout.convert_worker(fitness)
            hive.get_scouts().remove(scout)
            hive.get_workers().append(new_worker)

    return best_quality_food

In [10]:
best_food = bee_search(evaluate_roll,
                       minimize=False,
                       n_bees=30,
                       n_scouts=1,
                       max_iter=500,
                       limit=3)
print(f'Best price is : {best_food.location.total_price()}, best number of biscuits is :{best_food.location.number_of_biscuits()}')
print(f'Best food biscuit counts is : {best_food.location.biscuit_type_count()}')

Best price is : 657, best number of biscuits is :144
Best food biscuit counts is : {8: 33, 5: 8, 4: 29, 2: 23}


Biscuits are more uniformly distributed on the roll. Results are similar than Bogo optimizer but take longer to be executed.

The two first algorithms tend to have more homogenous distributions of biscuits with a high percentage of biscuit of size 8 and 4.  
The two last algorithms are more heterogenous and had a uniform repartition of biscuit types along the roll.
However we can infer adding randomness allows for better results, despite longer time of execution.